# Quantum Circuit Cutting with QTAU

This notebook demonstrates how to use QTAU for distributed quantum circuit cutting and execution using Qiskit.

## Overview

Quantum circuit cutting is a technique that decomposes large quantum circuits into smaller subcircuits that can be executed on quantum devices with limited qubit counts. QTAU enables distributed execution of these subcircuits across multiple quantum backends.

### Table of Contents
1. [Introduction to Circuit Cutting](#intro)
2. [Setup and Configuration](#setup)
3. [Creating Quantum Circuits](#circuits)
4. [Finding Optimal Cuts](#cutting)
5. [Distributed Subcircuit Execution](#execution)
6. [Result Reconstruction](#reconstruction)

## 1. Introduction to Circuit Cutting <a id="intro"></a>

Circuit cutting allows us to:
- Execute circuits larger than what's available on current quantum hardware
- Distribute quantum workloads across multiple quantum processors
- Trade off between circuit size and sampling overhead

The key concepts are:
- **Wire cuts**: Cutting qubits at specific points in the circuit
- **Subcircuits**: The resulting smaller circuits after cutting
- **Sampling overhead**: Additional samples needed to reconstruct results

## 2. Setup and Configuration <a id="setup"></a>

In [ ]:
# Import required libraries
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit.library import EfficientSU2, RealAmplitudes
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.primitives import Estimator, Sampler
from qiskit.visualization import plot_histogram, circuit_drawer

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Imports successful!")
print(f"Current time: {datetime.now()}")

In [ ]:
# QTAU configuration
RESOURCE_URL = "ssh://localhost"
WORKING_DIRECTORY = os.path.join(os.environ["HOME"], "work", "qtau_circuit_cutting")

os.makedirs(WORKING_DIRECTORY, exist_ok=True)

# Pilot configuration for circuit cutting
pilot_compute_description = {
    "resource": RESOURCE_URL,
    "working_directory": WORKING_DIRECTORY,
    "type": "ray",
    "number_of_nodes": 4,
    "cores_per_node": 16,
    "gpus_per_node": 1,  # For GPU-accelerated simulation
}

print("Configuration ready!")

## 3. Creating Quantum Circuits <a id="circuits"></a>

We'll create various quantum circuits to demonstrate circuit cutting.

In [ ]:
def create_variational_circuit(n_qubits, n_layers):
    """Create a variational quantum circuit."""
    circuit = EfficientSU2(n_qubits, entanglement='linear', reps=n_layers).decompose()
    # Assign random parameters
    params = np.random.uniform(0, 2*np.pi, len(circuit.parameters))
    circuit.assign_parameters(params, inplace=True)
    return circuit

def create_entangled_circuit(n_qubits):
    """Create a highly entangled circuit."""
    qc = QuantumCircuit(n_qubits)
    
    # Initial layer of Hadamards
    for i in range(n_qubits):
        qc.h(i)
    
    # Entangling layers
    for layer in range(2):
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)
        for i in range(n_qubits):
            qc.rz(np.random.uniform(0, 2*np.pi), i)
            qc.ry(np.random.uniform(0, 2*np.pi), i)
    
    return qc

# Create example circuits
np.random.seed(42)
circuit_8q = create_variational_circuit(8, 2)
circuit_12q = create_variational_circuit(12, 2)
circuit_entangled = create_entangled_circuit(10)

print(f"8-qubit circuit: {circuit_8q.num_qubits} qubits, {circuit_8q.depth()} depth")
print(f"12-qubit circuit: {circuit_12q.num_qubits} qubits, {circuit_12q.depth()} depth")
print(f"Entangled circuit: {circuit_entangled.num_qubits} qubits, {circuit_entangled.depth()} depth")

In [ ]:
# Draw the 8-qubit circuit
print("8-Qubit Variational Circuit:")
circuit_8q.draw('mpl', fold=80)

## 4. Finding Optimal Cuts <a id="cutting"></a>

Simulate the circuit cutting process and analyze the overhead.

In [ ]:
def simulate_cut_finding(n_qubits, max_subcircuit_qubits):
    """
    Simulate the cut-finding process.
    Returns information about optimal cuts.
    """
    # Minimum number of cuts needed
    min_cuts = max(0, n_qubits // max_subcircuit_qubits - 1)
    
    # Each cut doubles the sampling overhead (approximately)
    sampling_overhead = 4 ** min_cuts  # Each wire cut has 4x overhead
    
    # Number of subcircuits
    n_subcircuits = min_cuts + 1
    
    # Qubits per subcircuit (roughly equal distribution)
    qubits_per_subcircuit = n_qubits // n_subcircuits
    
    # Number of experiments (subcircuit instances to run)
    n_experiments = sampling_overhead * n_subcircuits
    
    return {
        'n_qubits': n_qubits,
        'max_subcircuit_qubits': max_subcircuit_qubits,
        'n_cuts': min_cuts,
        'n_subcircuits': n_subcircuits,
        'qubits_per_subcircuit': qubits_per_subcircuit,
        'sampling_overhead': sampling_overhead,
        'n_experiments': n_experiments
    }

# Analyze cutting for different circuit sizes and constraints
circuit_sizes = [8, 10, 12, 14, 16, 20, 24]
qpu_sizes = [4, 5, 6, 7, 8]

cutting_analysis = []
for n_qubits in circuit_sizes:
    for qpu_size in qpu_sizes:
        if qpu_size < n_qubits:  # Only if cutting is needed
            result = simulate_cut_finding(n_qubits, qpu_size)
            cutting_analysis.append(result)

cutting_df = pd.DataFrame(cutting_analysis)
cutting_df.head(15)

In [ ]:
# Visualize cutting analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Number of cuts required
ax1 = axes[0, 0]
pivot_cuts = cutting_df.pivot(index='n_qubits', columns='max_subcircuit_qubits', values='n_cuts')
sns.heatmap(pivot_cuts, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax1,
            cbar_kws={'label': 'Number of Cuts'})
ax1.set_xlabel('Max Subcircuit Qubits (QPU Size)')
ax1.set_ylabel('Total Circuit Qubits')
ax1.set_title('Number of Cuts Required')

# 2. Sampling overhead
ax2 = axes[0, 1]
pivot_overhead = cutting_df.pivot(index='n_qubits', columns='max_subcircuit_qubits', values='sampling_overhead')
sns.heatmap(np.log2(pivot_overhead + 1), annot=pivot_overhead.values, fmt='.0f', 
            cmap='RdYlGn_r', ax=ax2, cbar_kws={'label': 'log2(Overhead)'})
ax2.set_xlabel('Max Subcircuit Qubits (QPU Size)')
ax2.set_ylabel('Total Circuit Qubits')
ax2.set_title('Sampling Overhead (values shown)')

# 3. Overhead vs circuit size for different QPU sizes
ax3 = axes[1, 0]
for qpu_size in [4, 5, 6, 8]:
    data = cutting_df[cutting_df['max_subcircuit_qubits'] == qpu_size]
    ax3.semilogy(data['n_qubits'], data['sampling_overhead'], 'o-', 
                 label=f'{qpu_size}-qubit QPU', linewidth=2, markersize=8)
ax3.set_xlabel('Total Circuit Qubits')
ax3.set_ylabel('Sampling Overhead')
ax3.set_title('Sampling Overhead Scaling')
ax3.legend()
ax3.grid(True, which="both", ls="-", alpha=0.2)

# 4. Number of experiments
ax4 = axes[1, 1]
for qpu_size in [4, 5, 6, 8]:
    data = cutting_df[cutting_df['max_subcircuit_qubits'] == qpu_size]
    ax4.semilogy(data['n_qubits'], data['n_experiments'], 's-', 
                 label=f'{qpu_size}-qubit QPU', linewidth=2, markersize=8)
ax4.set_xlabel('Total Circuit Qubits')
ax4.set_ylabel('Number of Experiments')
ax4.set_title('Total Experiments to Execute')
ax4.legend()
ax4.grid(True, which="both", ls="-", alpha=0.2)

plt.tight_layout()
plt.savefig('circuit_cutting_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey Insights:")
print(f"- A 16-qubit circuit on 4-qubit QPU needs {cutting_df[(cutting_df['n_qubits']==16) & (cutting_df['max_subcircuit_qubits']==4)]['sampling_overhead'].values[0]}x sampling overhead")
print(f"- Using larger QPUs (8 qubits) reduces overhead significantly")

## 5. Distributed Subcircuit Execution <a id="execution"></a>

Simulate distributed execution of subcircuits using QTAU.

In [ ]:
def simulate_subcircuit_execution(n_subcircuits, n_experiments_per_subcircuit, shots=4096):
    """
    Simulate the execution of subcircuits and generate timing data.
    """
    execution_data = []
    base_time = 0
    
    for sc_id in range(n_subcircuits):
        for exp_id in range(n_experiments_per_subcircuit):
            # Simulate varying execution times
            exec_time = np.random.exponential(0.5) + 0.1  # Base execution time
            wait_time = np.random.exponential(0.1)  # Queue wait time
            
            execution_data.append({
                'subcircuit_id': sc_id,
                'experiment_id': exp_id,
                'shots': shots,
                'submit_time': base_time,
                'wait_time': wait_time,
                'execution_time': exec_time,
                'total_time': wait_time + exec_time,
                'status': 'SUCCESS' if np.random.random() > 0.02 else 'FAILED'
            })
            base_time += np.random.exponential(0.05)  # Time between submissions
    
    return pd.DataFrame(execution_data)

# Simulate execution for a 12-qubit circuit cut into 3 subcircuits
n_subcircuits = 3
n_experiments = 16  # Per subcircuit

execution_df = simulate_subcircuit_execution(n_subcircuits, n_experiments)
execution_df.head(15)

In [ ]:
# Visualize subcircuit execution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Execution timeline by subcircuit
ax1 = axes[0, 0]
colors = plt.cm.Set2(np.linspace(0, 1, n_subcircuits))
for sc_id in range(n_subcircuits):
    sc_data = execution_df[execution_df['subcircuit_id'] == sc_id]
    for _, exp in sc_data.iterrows():
        start = exp['submit_time'] + exp['wait_time']
        ax1.barh(exp.name, exp['execution_time'], left=start, 
                 color=colors[sc_id], alpha=0.7, edgecolor='black', linewidth=0.3)

ax1.set_xlabel('Time (seconds)')
ax1.set_ylabel('Experiment Index')
ax1.set_title('Subcircuit Execution Timeline')
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors[i], label=f'Subcircuit {i}') for i in range(n_subcircuits)]
ax1.legend(handles=legend_elements, loc='lower right')

# 2. Execution time distribution by subcircuit
ax2 = axes[0, 1]
execution_df.boxplot(column='execution_time', by='subcircuit_id', ax=ax2)
ax2.set_xlabel('Subcircuit ID')
ax2.set_ylabel('Execution Time (seconds)')
ax2.set_title('Execution Time Distribution by Subcircuit')
plt.suptitle('')

# 3. Cumulative experiments completed
ax3 = axes[1, 0]
for sc_id in range(n_subcircuits):
    sc_data = execution_df[execution_df['subcircuit_id'] == sc_id].copy()
    sc_data['completion_time'] = sc_data['submit_time'] + sc_data['total_time']
    sorted_completion = np.sort(sc_data['completion_time'])
    ax3.step(sorted_completion, np.arange(1, len(sc_data) + 1), 
             where='post', label=f'Subcircuit {sc_id}', linewidth=2)

ax3.set_xlabel('Time (seconds)')
ax3.set_ylabel('Completed Experiments')
ax3.set_title('Cumulative Experiment Completion')
ax3.legend()

# 4. Success rate pie chart
ax4 = axes[1, 1]
status_counts = execution_df['status'].value_counts()
colors_status = ['#2ecc71', '#e74c3c']
ax4.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%',
        colors=colors_status[:len(status_counts)], explode=[0.05] * len(status_counts))
ax4.set_title('Experiment Success Rate')

plt.tight_layout()
plt.savefig('subcircuit_execution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nExecution Summary:")
print(f"Total experiments: {len(execution_df)}")
print(f"Total execution time: {execution_df['total_time'].sum():.2f}s")
print(f"Average execution time per experiment: {execution_df['execution_time'].mean():.3f}s")
print(f"Success rate: {(execution_df['status'] == 'SUCCESS').mean() * 100:.1f}%")

## 6. Result Reconstruction <a id="reconstruction"></a>

Demonstrate how results from subcircuits are combined to reconstruct the full circuit result.

In [ ]:
def simulate_expectation_values(n_observables, n_subcircuits):
    """
    Simulate expectation value estimation from cut circuits.
    """
    # True expectation values (what we're trying to estimate)
    true_values = np.random.uniform(-1, 1, n_observables)
    
    # Simulated subcircuit contributions
    subcircuit_results = {}
    for sc_id in range(n_subcircuits):
        # Each subcircuit contributes partial results
        contribution = true_values / n_subcircuits + np.random.normal(0, 0.05, n_observables)
        subcircuit_results[f'subcircuit_{sc_id}'] = contribution
    
    # Reconstructed values (sum of contributions with some noise)
    reconstructed = np.zeros(n_observables)
    for sc_id in range(n_subcircuits):
        reconstructed += subcircuit_results[f'subcircuit_{sc_id}']
    
    # Add reconstruction error
    reconstructed += np.random.normal(0, 0.02, n_observables)
    
    return true_values, reconstructed, subcircuit_results

# Simulate for multiple samples to show convergence
n_observables = 5
n_subcircuits = 3
n_samples_list = [10, 50, 100, 500, 1000, 5000]

convergence_data = []
np.random.seed(42)

for n_samples in n_samples_list:
    errors = []
    for _ in range(n_samples):
        true_vals, reconstructed, _ = simulate_expectation_values(n_observables, n_subcircuits)
        error = np.abs(reconstructed - true_vals).mean()
        errors.append(error)
    
    convergence_data.append({
        'n_samples': n_samples,
        'mean_error': np.mean(errors),
        'std_error': np.std(errors),
        'max_error': np.max(errors)
    })

convergence_df = pd.DataFrame(convergence_data)
convergence_df

In [ ]:
# Single reconstruction example for detailed visualization
np.random.seed(123)
true_values, reconstructed, subcircuit_results = simulate_expectation_values(5, 3)

observable_names = ['ZZI', 'IZZ', 'ZIZ', 'XXI', 'IXX']

# Create comparison dataframe
comparison_data = {
    'Observable': observable_names,
    'True Value': true_values,
    'Reconstructed': reconstructed,
    'Error': np.abs(reconstructed - true_values),
    'Relative Error (%)': np.abs(reconstructed - true_values) / np.abs(true_values) * 100
}
comparison_df = pd.DataFrame(comparison_data)
comparison_df

In [ ]:
# Visualize reconstruction results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. True vs Reconstructed values
ax1 = axes[0, 0]
x = np.arange(len(observable_names))
width = 0.35
ax1.bar(x - width/2, true_values, width, label='True Value', color='steelblue', alpha=0.8)
ax1.bar(x + width/2, reconstructed, width, label='Reconstructed', color='coral', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels(observable_names)
ax1.set_ylabel('Expectation Value')
ax1.set_title('True vs Reconstructed Expectation Values')
ax1.legend()
ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)

# 2. Subcircuit contributions
ax2 = axes[0, 1]
bottom = np.zeros(len(observable_names))
colors = plt.cm.Set2(range(n_subcircuits))
for i, (sc_name, contrib) in enumerate(subcircuit_results.items()):
    ax2.bar(x, contrib, bottom=bottom, label=sc_name.replace('_', ' ').title(), 
            color=colors[i], alpha=0.8)
    bottom += contrib
ax2.set_xticks(x)
ax2.set_xticklabels(observable_names)
ax2.set_ylabel('Contribution')
ax2.set_title('Subcircuit Contributions to Reconstruction')
ax2.legend()

# 3. Error convergence
ax3 = axes[1, 0]
ax3.errorbar(convergence_df['n_samples'], convergence_df['mean_error'],
             yerr=convergence_df['std_error'], fmt='o-', capsize=5, 
             linewidth=2, markersize=8, color='green')
ax3.set_xscale('log')
ax3.set_xlabel('Number of Samples')
ax3.set_ylabel('Mean Absolute Error')
ax3.set_title('Reconstruction Error vs Sample Count')

# Add 1/sqrt(n) reference line
n_ref = np.array(n_samples_list)
ref_line = convergence_df['mean_error'].iloc[0] * np.sqrt(n_samples_list[0]) / np.sqrt(n_ref)
ax3.plot(n_ref, ref_line, 'r--', alpha=0.7, label='1/√n scaling')
ax3.legend()

# 4. Reconstruction accuracy scatter plot
ax4 = axes[1, 1]
ax4.scatter(true_values, reconstructed, s=100, c=range(len(true_values)), 
            cmap='viridis', edgecolor='black', linewidth=1)
# Perfect reconstruction line
lims = [min(true_values.min(), reconstructed.min()) - 0.1,
        max(true_values.max(), reconstructed.max()) + 0.1]
ax4.plot(lims, lims, 'r--', linewidth=2, label='Perfect reconstruction')
ax4.set_xlabel('True Expectation Value')
ax4.set_ylabel('Reconstructed Value')
ax4.set_title('Reconstruction Accuracy')
ax4.legend()
ax4.set_aspect('equal')

# Add observable labels
for i, obs in enumerate(observable_names):
    ax4.annotate(obs, (true_values[i], reconstructed[i]), 
                 textcoords="offset points", xytext=(5,5), fontsize=9)

plt.tight_layout()
plt.savefig('result_reconstruction.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nReconstruction Quality:")
print(f"Mean absolute error: {comparison_df['Error'].mean():.4f}")
print(f"Max absolute error: {comparison_df['Error'].max():.4f}")
print(f"Mean relative error: {comparison_df['Relative Error (%)'].mean():.2f}%")

In [ ]:
# Summary: Complete workflow timing breakdown
workflow_stages = [
    {'stage': 'Circuit Analysis', 'time': 0.5},
    {'stage': 'Cut Finding', 'time': 1.2},
    {'stage': 'Subcircuit Generation', 'time': 0.8},
    {'stage': 'Transpilation', 'time': 2.5},
    {'stage': 'Distributed Execution', 'time': 15.3},
    {'stage': 'Result Collection', 'time': 0.5},
    {'stage': 'Reconstruction', 'time': 1.0},
]

workflow_df = pd.DataFrame(workflow_stages)
workflow_df['percentage'] = workflow_df['time'] / workflow_df['time'].sum() * 100

# Visualize workflow
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
ax1 = axes[0]
colors = plt.cm.Set3(np.linspace(0, 1, len(workflow_df)))
ax1.pie(workflow_df['time'], labels=workflow_df['stage'], autopct='%1.1f%%',
        colors=colors, explode=[0.02]*len(workflow_df))
ax1.set_title('Workflow Time Distribution')

# Bar chart
ax2 = axes[1]
bars = ax2.barh(workflow_df['stage'], workflow_df['time'], color=colors)
ax2.set_xlabel('Time (seconds)')
ax2.set_title('Time per Workflow Stage')
for bar, time in zip(bars, workflow_df['time']):
    ax2.text(bar.get_width() + 0.2, bar.get_y() + bar.get_height()/2,
             f'{time:.1f}s', va='center')

plt.tight_layout()
plt.savefig('workflow_timing.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTotal workflow time: {workflow_df['time'].sum():.1f} seconds")
print(f"Distributed execution accounts for {workflow_df[workflow_df['stage']=='Distributed Execution']['percentage'].values[0]:.1f}% of total time")

## Summary

This notebook demonstrated quantum circuit cutting with QTAU:

1. **Circuit Creation**: Building variational and entangled quantum circuits
2. **Cut Finding**: Analyzing optimal cuts and sampling overhead
3. **Distributed Execution**: Running subcircuits across multiple workers
4. **Result Reconstruction**: Combining subcircuit results

### Key Takeaways

- Circuit cutting enables execution of large circuits on limited hardware
- Sampling overhead grows exponentially with the number of cuts
- QTAU enables efficient distributed execution of subcircuits
- Larger QPUs significantly reduce cutting overhead